In [160]:
import math
import random
import numpy as np
import pandas as pd
from datetime import datetime


import tensorflow as tf
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()


from tqdm import tqdm_notebook, tqdm
from collections import deque

#Warning ignore
import warnings
warnings.filterwarnings('ignore')

In [161]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    # Define hyperparamaters
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
        
    # Call a function  to build a model trought this class constructor
    # More parameters could be ustilized to programaticaly define network size (layers and neurons)
    self.model = self.model_builder()
    
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))    
    return model
  
  
  
  
  # Trade function that takes state as an input and returns an action 
  # to perform in perticular state 
  def trade(self, state):
    
    # Should we perform a renadom generated action or action defined in model?
    
    # If value from our random generator is smaller or equal to our epsilon 
    #     then we will retun a random action from action_space [0-3)
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    # If our random is greater than epsilon then we will use model to perform action
    actions = self.model.predict(state)
    # return only a one number defining an action (#Stay - 0 , Buy - 1, Sell - 2) 
    #    that has maximum probability
    return np.argmax(actions[0])
  
  
  
  def batch_train(self, batch_size):
    
    batch = []
    
    # Iterrate in momory, we do not want to randolmy select data as we are dealing with 
    #    time constraint data. We will always sample from the end of memory size of bath
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      # insert data from memory to batch      
      batch.append(self.memory[i])
    
    
    # Iterate trought batch of data and train the model for each sample from batch
    # Order of variables in for loop is important
    for state, action, reward, next_state, done in batch:
      # Reward if agent is in terminal state
      reward = reward
      # Check that agent is not in terminal state
      # If not in terminal state calculate reward for actions that could be played
      if not done:
        # Discounted total reward:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])        
      # Target variable that is predicted by the model (action)
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    # We will decrease epsilon parameter that is 1 as defined in __init__  so
    #    so we can stop performing random actions at some point
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [244]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

  
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))


def dataset_loader(stock_name):
  
  #Use pandas data reader for reading stock data from warious sources like "yahoo", "google"
  dataset = pdr.get_data_yahoo("SPY", start="2017-01-01", end="2024-01-01")
   
  # Get start and end time to variables from dataset
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  # Model will use "Close" column for training 
  close = dataset['Close']

  return close

# Data -> dataset to predict from, gathered by data:loader()
# Timestep -> Day in the dataset that we want to predict for [0:datalength]
# window_suze -> how many days in past we want to use to predict current status[1:datalength]
#         Try different setup to see what creates best fit
def state_creator(data, timestep, window_size):
  
  # starting day of our state
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    # Replicate member (data[0]) needed times
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  # Iterate trough whole windowed_data minus current state (-1)
  for i in range(window_size - 1):
    # Normalize the difference from current day and the next day
    # Because the prices can be very different and we want them on same scale
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])


In [236]:
## Carga los datos utilizados para los indicadores
data = pd.read_parquet('resultadoIndicadores.parquet')
data = data.reset_index()
data = data['SMA_21']

In [239]:
window_size = 10
episodes = 1000 # same as epoch

batch_size = 32
data_samples = len(data) - 1 # discard last value, that we will predict on

In [241]:
trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 32)                352       
                                                                 
 dense_25 (Dense)            (None, 64)                2112      
                                                                 
 dense_26 (Dense)            (None, 128)               8320      
                                                                 
 dense_27 (Dense)            (None, 3)                 387       
                                                                 
Total params: 11171 (43.64 KB)
Trainable params: 11171 (43.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
# Tage data for Apple ------------------------
stock_name = "AAPL"
data = dataset_loader(stock_name)

[*********************100%%**********************]  1 of 1 completed


In [151]:
trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 32)                352       
                                                                 
 dense_21 (Dense)            (None, 64)                2112      
                                                                 
 dense_22 (Dense)            (None, 128)               8320      
                                                                 
 dense_23 (Dense)            (None, 3)                 387       
                                                                 
Total params: 11171 (43.64 KB)
Trainable params: 11171 (43.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [252]:
state = state_creator(data, 0, window_size + 1)
total_profit = []
trader.inventory = []


for t in tqdm(range(10)):
    action = trader.trade(state)

        # Use action to get to next state(t+)
    next_state = state_creator(data=data, timestep=(t + 1), window_size=(window_size + 1))
    # As we did not calculate anything up to this point reward is 0
    reward = 0
    
    if action == 1: #Buying
      # Put buyed stock to inventory to trade with
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    # To sell we need to have something in inventory  
    elif action == 2 and len(trader.inventory) > 0: #Selling
      # Check buy price, pop removes first value from list
      buy_price = trader.inventory.pop(0)
      
      # If we gain money (current price - buy price) we have reward 
      #    if we lost money then reward is 0
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    # if t is last sample in our dateset we are done
    #     we do not have any steps to perform in current episode
    if t == data_samples - 1:
      done = True
    else:
      done = False
    
    # Append all data to trader-agent memory, experience buffer
    trader.memory.append((state, action, reward, next_state, done))
    
    # change state to next state, so we are done with an episode
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    # Chekc if we have more information in our memory than batch size
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
  
  # Save the model every 10 episodes
    if episode % 10 == 0:
       trader.model.save("ai_trader_{}.h5".format(episode))


  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 18ms/step


 10%|█         | 1/10 [00:04<00:43,  4.86s/it]

1/1 [==============================] - 0s 16ms/step


 20%|██        | 2/10 [00:09<00:38,  4.86s/it]

...

 80%|████████  | 8/10 [00:37<00:09,  4.57s/it]

1/1 [==============================] - 0s 16ms/step


 90%|█████████ | 9/10 [00:41<00:04,  4.54s/it]

1/1 [==============================] - 0s 16ms/step


100%|██████████| 10/10 [00:46<00:00,  4.61s/it]


In [253]:
trader.memory

deque([(array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.33071239, 0.36041518]]),
        1,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.33071239]]),
        False),
       (array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.33071239]]),
        0,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.33071239]]),
        False),
       (array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.33071239]]),
        0,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.33071239,

In [91]:
batch_size

32

In [92]:
trader.memory[0]

(array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]),
 2,
 0,
 array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
         0.5       , 0.5       , 0.5       , 0.5       , 0.33071239]]),
 False)

In [93]:
data

0      146.202915
1      145.497950
2      144.924388
3      145.094980
4      145.688369
          ...    
881    325.840001
882    325.112858
883    324.418097
884    323.774286
885    323.648096
Name: SMA_21, Length: 886, dtype: float64

In [94]:
state

array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.33071239]])

In [255]:
trader.model.predict(state)

1/1 [==============================] - 0s 17ms/step


array([[3.4609067, 3.483146 , 3.84108  ]], dtype=float32)

In [96]:
state

array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       , 0.5       , 0.5       , 0.5       , 0.33071239]])

In [97]:
trader.trade(state)

2

In [98]:
trader.memory

deque([(array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]]),
        2,
        0,
        array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
                0.5       , 0.5       , 0.5       , 0.5       , 0.33071239]]),
        False)],
      maxlen=2000)